In [4]:
import pysam 
import pandas as pd

In [10]:
PATH_genome = '/node200data/18parkky/datasets/reference/gd/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa'
PATH_gff    = '/node200data/18parkky/datasets/reference/gd/Arabidopsis_thaliana.TAIR10.56.gtf'

def LoadGencodeGFF( PATH_to_GencodeGFF, featureOfInterest='exon', skiprows_n=5):
    GencodeGFF = pd.read_csv(PATH_to_GencodeGFF, sep='\t', skiprows=skiprows_n, header=None)
    GencodeGFF.columns = ['sequence', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']

    GencodeGFF.dropna(inplace=True)
    if featureOfInterest == None:
        pass 
    else:
        GencodeGFF = GencodeGFF[(GencodeGFF['feature']==featureOfInterest)]
        
    col1, col2, col3, col4 = list(), list(), list(), list()

    for tup in GencodeGFF.itertuples():
        try:
            attributes = tup.attributes.split(';')
        except:
            print(tup)
            raise ValueError
        
        ENSG, ENST = None, None
        GeneName, TranscriptName = None, None
        
        for attribute in attributes:
            if 'gene_id' in attribute:
                ENSG = attribute.strip().split(" ")[1].split('\"')[1]
                col1.append(ENSG)
                
            elif 'transcript_id' in attribute:
                ENST = attribute.strip().split(" ")[1].split('\"')[1]
                col2.append(ENST)

            elif 'gene_name' in attribute:
                GeneName = attribute.strip().split(" ")[1].split('\"')[1].upper()
                col3.append(GeneName)

            elif 'transcript_name' in attribute:
                TranscriptName = attribute.strip().split(" ")[1].split('\"')[1].upper()
                col4.append(TranscriptName)

        if ENSG == None:
            col1.append('N/A')
        if ENST == None:
            col2.append('N/A')
        if GeneName == None:
            col3.append('N/A')
        if TranscriptName == None:
            col4.append('N/A')
        
    GencodeGFF['ENSG'] = col1
    GencodeGFF['ENST'] = col2
    GencodeGFF['GeneName'] = col3
    GencodeGFF['TranscriptName'] = col4
    
    return GencodeGFF

gff = LoadGencodeGFF( PATH_gff, featureOfInterest=None, )

/tmp/ipykernel_222102/3242569405.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  GencodeGFF = pd.read_csv(PATH_to_GencodeGFF, sep='\t', skiprows=skiprows_n, header=None)


In [3]:
genome = pysam.FastaFile(PATH_genome)

In [15]:
with open(f'/node200data/18parkky/datasets/reference/gd/CustomTranscriptome.fa', 'w') as fa:
    for ENST, edf in gff[gff['feature']=='exon'].groupby('ENST'):
        ENSG_e              = edf.iloc[0].ENSG
        GeneName_e          = edf.iloc[0].GeneName
        TranscriptName_e    = edf.iloc[0].TranscriptName
        EntryName = f'{ENST}|{ENSG_e}|{ENST}|{ENSG_e}|{TranscriptName_e}|{GeneName_e}|1|1'
        
        chrom = edf.iloc[0].sequence
        
        seq = str()
        for tup in edf.sort_values('start').itertuples():
            for nt in genome.fetch( str(tup.sequence), tup.start-1, tup.end ): seq += nt.upper()
        
        fa.write(f'>{EntryName}\n')
        fa.write(f'{seq}\n')

In [1]:
def SummarizeAlignment( bamfile, References, GencodeGFF, DIR_out, filename, processNumber ):
    
    AlignmentSummary = list()

    for reference in References:

        for read in bamfile.fetch(reference, multiple_iterators=True):
            
            ReadName = read.query_name.split('|')[0]
            CellType = read.query_name.split('|')[1]
            CB, UMI  = read.query_name.split('|')[2], read.query_name.split('|')[3]
            
            ENSMUST = read.reference_name.split('|')[0]
            ENSMUSG = read.reference_name.split('|')[1]
            GeneName = read.reference_name.split('|')[5]
            
            GFF_oi = GencodeGFF[GencodeGFF['ENST']==ENSMUST.split('.')[0]].sort_values('start')
            GFF_oi['length'] = GFF_oi['end'] - GFF_oi['start'] + 1
            GFF_oi.reset_index(inplace=True, drop=True)

            if len(GFF_oi)<=1: continue 
            
            dict_ExonNumber_to_PERC     = dict()
            AlignedReferencePositions   = set(read.get_reference_positions())
            
            for idx, tup in enumerate(GFF_oi.itertuples()):
                if idx == 0:
                    ExonRelativeStartPos    = 1
                    ExonRelativeEndPos      = int(tup.length)
                else:
                    ExonRelativeStartPos    = int(ExonRelativeEndPos + 1)
                    ExonRelativeEndPos      = int(ExonRelativeStartPos + tup.length)
                    
                uncoveredPosCount = 0
                    
                for pos in range(ExonRelativeStartPos, ExonRelativeEndPos):
                    if pos in AlignedReferencePositions:
                        pass 
                    else:
                        uncoveredPosCount += 1
                
                PERC = 100*(1-uncoveredPosCount/tup.length)
                dict_ExonNumber_to_PERC[idx] = PERC
                

            AlignmentSummary.append( [ReadName, CB, UMI, CellType, 
                                    GeneName, ENSMUSG, ENSMUST, 
                                    read.mapping_quality, read.flag, read.cigarstring,
                                    sum(list(dict_ExonNumber_to_PERC.values())), np.mean(list(dict_ExonNumber_to_PERC.values()))] )
        
    AlignmentSummary = pd.DataFrame(AlignmentSummary, columns=['ReadName', 'CB', 'UMI', 'CellType', 
                                                               'GeneName', 'ENSMUSG', 'ENSMUST', 
                                                               'MAPQ', 'flag', 'cigarstring',
                                                               'SumOfPERC', 'AvgOfPERC', ])
    
    return AlignmentSummary
    # AlignmentSummary.to_csv(f'{DIR_out}/{filename}.SummarizeAlignment.temp{processNumber}.tsv', sep='\t', index=False)

In [5]:
PATH_BAM    = '/node200data/18parkky/datasets/reference/gd/arabidopsis_run1_2_realigned_filtered.bam'
bamfile     = pysam.AlignmentFile(PATH_BAM, 'rb')

[W::hts_idx_load3] The index file is older than the data file: /node200data/18parkky/datasets/reference/gd/arabidopsis_run1_2_realigned_filtered.bam.bai


In [9]:
import math 
threads = 72

ChunkSize = math.ceil( len(bamfile.references)/threads )
ReferenceIsoforms_chunks = [ bamfile.references[i:i+ChunkSize] for i in range(0, len(bamfile.references), ChunkSize) ]

In [17]:
import numpy as np 

AlignmentSummary = list()

for reference in ReferenceIsoforms_chunks[0]:

    for read in bamfile.fetch(reference, multiple_iterators=True):
        
        ReadName = read.query_name.split('|')[0]
        CellType = read.query_name.split('|')[1]
        CB, UMI  = read.query_name.split('|')[2], read.query_name.split('|')[3]
        
        ENSMUST = read.reference_name.split('|')[0]
        ENSMUSG = read.reference_name.split('|')[1]
        GeneName = read.reference_name.split('|')[5]
        
        print(ENSMUST)
        
        
        GFF_oi = gff[gff['ENST']==ENSMUST].sort_values('start')
        GFF_oi['length'] = GFF_oi['end'] - GFF_oi['start'] + 1
        GFF_oi.reset_index(inplace=True, drop=True)

        if len(GFF_oi)<=1: 
            continue 
        
        dict_ExonNumber_to_PERC     = dict()
        AlignedReferencePositions   = set(read.get_reference_positions())
        
        for idx, tup in enumerate(GFF_oi.itertuples()):
            if idx == 0:
                ExonRelativeStartPos    = 1
                ExonRelativeEndPos      = int(tup.length)
            else:
                ExonRelativeStartPos    = int(ExonRelativeEndPos + 1)
                ExonRelativeEndPos      = int(ExonRelativeStartPos + tup.length)
                
            uncoveredPosCount = 0
                
            for pos in range(ExonRelativeStartPos, ExonRelativeEndPos):
                if pos in AlignedReferencePositions:
                    pass 
                else:
                    uncoveredPosCount += 1
            
            PERC = 100*(1-uncoveredPosCount/tup.length)
            dict_ExonNumber_to_PERC[idx] = PERC
            

        AlignmentSummary.append( [ReadName, CB, UMI, CellType, 
                                GeneName, ENSMUSG, ENSMUST, 
                                read.mapping_quality, read.flag, read.cigarstring,
                                sum(list(dict_ExonNumber_to_PERC.values())), np.mean(list(dict_ExonNumber_to_PERC.values()))] )
        
    break
AlignmentSummary = pd.DataFrame(AlignmentSummary, columns=['ReadName', 'CB', 'UMI', 'CellType', 
                                                            'GeneName', 'ENSMUSG', 'ENSMUST', 
                                                            'MAPQ', 'flag', 'cigarstring',
                                                            'SumOfPERC', 'AvgOfPERC', ])

AT1G01010.1


In [19]:
AlignmentSummary

,ReadName,CB,UMI,CellType,GeneName,ENSMUSG,ENSMUST,MAPQ,flag,cigarstring,SumOfPERC,AvgOfPERC
0,05fb7733-3e93-4669-91c9-294162dcce05_R_0,AGTACCACAAGTATCC,AAGTCCGTACAT,2,NAC001,AT1G01010,AT1G01010.1,60,0,3S1476M2D72M24S,68.267959,4.015762
